## Spots that are always excluded:
##### Test spots
##### Spots marked as problematic
##### Suscpicious for analysis == 'Yes' spots
##### Fake Sale spots: This means that Reason for Cancelling Survey is any combination which contains 'Fake Sale'
##### Spots whose start date was before 2018


## CAN CANCEL:
##### Spots which cancelled more than 2 months before their Available Cancellation Date are excluded
##### Spots whose Available Cancellation Date is 2 or more months after the date of analysis
##### Months during which spots couldn't have cancelled according to the above conditions

## wo CB sets:
##### Closed Business, Sold Business and Non-payment spots are excluded. This means Reason for Cancelling Survey is any combination which contains 'Closed Business', 'Closed/Sold Business', 'Sold Business', 'Sold/Closed Business' or 'Non-payment'
## canc conf event
##### Spots for which Date Cancellation Confirmed is more than 60 days after Date Cancellation Requested

#### ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import math
import importlib
from dateutil.relativedelta import relativedelta
import get_stopped_doing_something_variables
import prepare_for_the_models
import fit_tv_cox_models
import read_a_combination_of_variables
import yaml
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, Markdown

#### Helpers ####
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from helpers.s3_bucket_utils import S3BucketUtils
from helpers import db_utils
from helpers import settings

bucket = S3BucketUtils()
################

import calculate_all_vars_for_churn_prediction
import churn_risk_calculation_test
import churn_risk_calculation
import model_testing

In [2]:
# read the yaml file with a list of parameters needed for the report
with open(r'./parameters/stopped_doing_something_report_parameters.yaml') as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)

date_of_analysis = parameters['date_of_analysis']
date_dir = date_of_analysis.replace('-', '_')
### name of the data directory ###
churn_based_on_behaviour_dir = parameters['churn_based_on_behaviour_dir']
hs_list_filename = parameters['hs_list_filename']
hs_list_path = 'churn_analysis/data/'+date_dir+'/'+hs_list_filename

prediction_months = parameters['prediction_months']
### penalizer value when fitting the models ###
penalizer = parameters['penalizer']
### model type ###
model_type = parameters['model_type']
### coefficient and p values when dropping unsignificant variables ###
coefficient_limit_for_numerical_vars = parameters['coefficient_limit_for_numerical_vars']
coefficient_limit_for_cat_vars = parameters['coefficient_limit_for_cat_vars']
p_limit = parameters['p_limit']
additional_higher_p_limit = parameters['additional_higher_p_limit']
additional_lower_p_limit = parameters['additional_lower_p_limit']

# read the yaml file with data set parameters #
# data_set_name = input('Data set name: ')
data_set_name = 'CAN_CANCEL_spots_wo_CB_cancellation_requested'
with open(r'./parameters/data_sets.yaml') as file:
    data_sets_parameters = yaml.load(file, Loader=yaml.FullLoader)
    
spots_set = data_sets_parameters[data_set_name]['spots_set']
with_wo_CB = data_sets_parameters[data_set_name]['with_wo_CB']
event_date_full_name = data_sets_parameters[data_set_name]['event_date_type']

if with_wo_CB == 'with_CB':
    with_wo_CB_boolean = True
else:
    with_wo_CB_boolean = False

if event_date_full_name == 'cancellation_requested':
    event_date = 'canc_req'
elif event_date_full_name == 'cancellation_confirmed':
    event_date = 'canc_conf'

In [3]:
##### base columns ##### 
base_cols = ['spot_id',\
            'time',\
            'event']

In [4]:
##### Model 0: all variables - prepare data for the model ####
cols_to_use = read_a_combination_of_variables.\
main(model_number=0, dir_name='combinations_of_variables_that_are_not_dependent/')
#### get behavioural variables ####
(variables_to_use_for_the_model, did_something_before_vars, did_something_last_month_vars) = \
get_stopped_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
cols = base_cols + variables_to_use_for_the_model + \
did_something_before_vars + did_something_last_month_vars

data_all_spots = \
calculate_all_vars_for_churn_prediction.\
get_data_for_the_MV_Cox_model(date_of_analysis=date_of_analysis,\
                              hs_filename=hs_list_filename,\
                              spots_set=spots_set,\
                              with_wo_CB=with_wo_CB,\
                              event_date=event_date, columns=cols,\
                              data_dir=churn_based_on_behaviour_dir)

#### get data for the model ###
(data, base_df, df_timeline_all_vars) = \
prepare_for_the_models.get_data_for_the_MV_Cox_model(date_of_analysis=date_of_analysis, spots_set=spots_set, \
                            with_wo_CB=with_wo_CB, event_date=event_date, columns=cols, data_dir=churn_based_on_behaviour_dir,\
                            C = 100)
    
model_numbers = \
read_a_combination_of_variables.get_a_list_of_model_numbers(dir_name='combinations_of_variables_that_are_not_dependent/')

In [5]:
# properly_used_inquiries_models = read_a_combination_of_variables.\
# get_properly_used_inquiries_model_names_and_numbers(dir_name='combinations_of_variables_that_are_not_dependent/')

In [6]:
## determine which models are not already fit ##
import models_to_be_fit
num_of_p_values = 1
num_of_prediction_months = len(prediction_months)
churn_risk_prediction_exports = 'data/'+date_dir+'/exports/churn_risk_prediction/'+model_type+'/'
coefficients_and_pvalues_exports = 'data/'+date_dir+'/exports/coefficients_and_pvalues/'+model_type+'/'
(already_fit, yet_to_be_fit) = models_to_be_fit.main(model_numbers=model_numbers,\
                                                     model_type=model_type,\
                                                     date_dir=date_dir,\
                                                     churn_risk_prediction_exports=churn_risk_prediction_exports,\
                                                     coefficients_and_pvalues_exports=coefficients_and_pvalues_exports,\
                                                     data_set_name=data_set_name,\
                                                     num_of_p_values=num_of_p_values,\
                                                     num_of_prediction_months=num_of_prediction_months)

# for model_number in properly_used_inquiries_models['model_number'].unique():
for model_number in yet_to_be_fit:
    cols_to_use = read_a_combination_of_variables.\
    main(model_number=model_number, dir_name='combinations_of_variables_that_are_not_dependent/')
    
    model_name = \
    read_a_combination_of_variables.get_model_names(model_number=model_number, \
                                                    dir_name='combinations_of_variables_that_are_not_dependent/')
    display(Markdown("# Model "+ str(model_number) + ": " + model_name))

    #### get behavioural variables ####
    (variables_to_use_for_the_model, did_something_before_vars, did_something_last_month_vars) = \
    get_stopped_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
    cols = base_cols + variables_to_use_for_the_model + \
    did_something_before_vars + did_something_last_month_vars

    #### data for the model ###
    df_timeline = df_timeline_all_vars.copy()
    vars_that_stay = ['spot_id', 'start', 'stop', 'event']+\
    [x for x in cols if x not in base_cols]+\
    [x for x in df_timeline.columns if 'spot_category_' in x or 'metro_area_' in x]
    df_timeline.drop([x for x in df_timeline.columns if x not in vars_that_stay], axis = 1, inplace = True)

    ### variables to skip ###
    df_timeline.isnull().sum().sum() #OK
    skip_vars = list((df_timeline!=0).sum()[(df_timeline!=0).sum()==0].index)
    skip_vars

    ctv = fit_tv_cox_models.fit_the_models_and_print_summaries(df_timeline=df_timeline, base_df=base_df, \
                                                         date_of_analysis=date_of_analysis, model_type=model_type, \
                                                         variables_to_use_for_the_model=cols_to_use,\
                                                         coefficient_limit_for_numerical_vars=coefficient_limit_for_numerical_vars, \
                                                         coefficient_limit_for_cat_vars=coefficient_limit_for_cat_vars, p_limit=p_limit, \
                                                         additional_higher_p_limit=additional_higher_p_limit, \
                                                         additional_lower_p_limit=additional_lower_p_limit,\
                                                         skip_vars=skip_vars, penalizer=penalizer)

    ### save coefs and p values for p < 0.2 ###
    fit_tv_cox_models.save_results(df = ctv.summary.reset_index(), date_of_analysis = date_of_analysis, \
                                   data_dir = churn_based_on_behaviour_dir, dir_name='exports/coefficients_and_pvalues',\
                 results_name = 'coef_and_pvalues', spots_set = spots_set,\
                 with_wo_CB = with_wo_CB, event_date_type = event_date_full_name, p_limit=0.2, model_number=model_number,\
                                  model_type = model_type)
    
    churn_risk_calculation.calculate_churn_risk(date_of_analysis = date_of_analysis, df_timeline = df_timeline, \
                                            vars_ = list(ctv.summary.index), data_all_spots = data_all_spots, ctv = ctv, \
                                            model_type = model_type, model_number = model_number,\
                                            with_add_vars = False, p_limit = p_limit,\
                                            event_date_type=event_date_full_name, spots_set = spots_set, with_wo_CB=with_wo_CB)
    
    for prediction_month in prediction_months:
        testing_results = model_testing.\
        get_testing_results(date_of_analysis=date_of_analysis,\
                            df_timeline=df_timeline.copy(), \
                            base_df = base_df.copy(),\
                            data_all_spots=data_all_spots.copy(), \
                            hs_list_path=hs_list_path,\
                            model_type=model_type,\
                            model_number=model_number,\
                            with_add_vars=False, p_limit=p_limit, event_date_type=event_date_full_name, \
                            prediction_month=prediction_month, ctv=ctv, spots_set=spots_set, \
                            with_wo_CB=with_wo_CB, penalizer=penalizer, test_with_req_canc_before=True)

        model_testing.save_results(date_of_analysis=date_of_analysis,\
                               df=testing_results[0], \
                               results_name='testing_results', \
                               model_type=model_type,\
                               model_number=model_number,\
                               spots_set=spots_set,\
                               with_wo_CB=with_wo_CB,\
                               event_date_type=event_date_full_name,\
                               with_add_vars=False, p_limit=p_limit, prediction_month=prediction_month)
        
        display(Markdown("## Model "+ str(model_number) + ": testing results"))
        display(testing_results[0].set_index('index'))

# Model 42: Opted.out.of.facebook

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 32630 periods, 2550 subjects, 282 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 2550
 number of periods = 32630
  number of events = 282
partial log-likelihood = -1761.42
  time fit was run = 2022-11-05 00:31:59 UTC

---
                                                               coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                            
Annual Commitment with 2 semi-annual payments                 -0.29       0.75       0.41            -1.09             0.51                 0.34                 1.67
Annual Prepay                                                  0.52       1.68       0.15             0.22             0.81                 1.25                 2.24
Monthly                                                        0.51       1.67       0.27            -0.03             1.05                 0.97                 2.86
New.email.subscriber.signups.last.month.div10                  0.00       1.00       0.04            -0.08             0.08                 0.92                 1.09
New.email.subscribers.allinclusive.last.month.div10           -0.00       1.00       0.02            -0.05             0.04                 0.95                 1.04
Opted.out.of.facebook                                         -0.14       0.87       0.25            -0.63             0.36                 0.53                 1.43
SpotHopper_site_not_live_yet                                   0.71       2.03       0.33             0.05             1.37                 1.05                 3.92
Website.Views.last.month.total.log2                           -0.21       0.81       0.05            -0.32            -0.11                 0.73                 0.90
had_added_events_manually_edited_events_before                -0.02       0.98       0.18            -0.36             0.33                 0.70                 1.39
had_added_events_manually_edited_events_last_4_months         -0.36       0.70       0.25            -0.84             0.12                 0.43                 1.13
had_added_food_edited_food_before                              0.04       1.05       0.17            -0.29             0.38                 0.75                 1.46
had_added_food_edited_food_last_4_months                      -0.11       0.89       0.19            -0.48             0.26                 0.62                 1.29
had_added_specials_edited_specials_before                     -0.15       0.86       0.15            -0.45             0.14                 0.64                 1.15
had_added_specials_edited_specials_last_4_months              -0.13       0.88       0.23            -0.58             0.32                 0.56                 1.38
had_bday_club_downloads_before                                 0.19       1.21       0.61            -1.01             1.39                 0.36                 4.02
had_bday_club_downloads_last_2_months                          0.47       1.61       0.70            -0.90             1.85                 0.40                 6.37
had_catering_submissions_before                                0.15       1.16       0.23            -0.29             0.60                 0.75                 1.82
had_catering_submissions_last_4_months                        -0.22       0.80       0.24            -0.69             0.24                 0.50                 1.27
had_clicked_emails_before                                      0.44       1.56       0.17             0.12             0.77                 1.13                 2.16
had_clicked_emails_last_4_months                              -0.33       0.72       0.15            -0.63            -0.03                 0.53                 0.97
had_consumer_job_listing_inquiries_before  

## without categorical variables with |coeff| < 0.2 and p value > 0.2

## without numerical variables with |coeff| < 0.01 and p value > 0.2

...

,all_spots_that_requested_cancellation,all_spots_that_requested_cancellation_wo_Closed_Sold_Business
index,,
total_number_of_spots,3165,3129
requested_cancellation,19,11
prediction_month,2022-08-01,2022-08-01
pearsons_corr_coef_and_p_value,"(0.067, 0.0)","(0.07, 0.0)"
spearmans_corr_coef_and_p_value,"(0.049, 0.006)","(0.055, 0.002)"
log_loss,0.041682,0.022948
concordance,68.366,76.789
discordance,31.634,23.211
requested_cancellation_in_the_first_20_%_of_all_spots,10,6


## Model 45: testing results

,all_spots_that_requested_cancellation,all_spots_that_requested_cancellation_wo_Closed_Sold_Business
index,,
total_number_of_spots,3403,3374
requested_cancellation,39,25
prediction_month,2022-09-01,2022-09-01
pearsons_corr_coef_and_p_value,"(0.117, 0.0)","(0.125, 0.0)"
spearmans_corr_coef_and_p_value,"(0.055, 0.001)","(0.086, 0.0)"
log_loss,0.07756,0.044652
concordance,64.995,79.039
discordance,35.005,20.961
requested_cancellation_in_the_first_20_%_of_all_spots,19,16


## Model 45: testing results

,all_spots_that_requested_cancellation,all_spots_that_requested_cancellation_wo_Closed_Sold_Business
index,,
total_number_of_spots,3692,3677
requested_cancellation,44,29
prediction_month,2022-10-01,2022-10-01
pearsons_corr_coef_and_p_value,"(0.062, 0.0)","(0.024, 0.15)"
spearmans_corr_coef_and_p_value,"(0.106, 0.0)","(0.09, 0.0)"
log_loss,0.071588,0.048443
concordance,78.312,79.232
discordance,21.688,20.768
requested_cancellation_in_the_first_20_%_of_all_spots,23,16
